In [109]:
import pandas as pd
import time
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

import warnings
warnings.filterwarnings('ignore')

driver = webdriver.Chrome('/Users/ziga/Downloads/chromedriver')

In [113]:
from selenium.webdriver.chrome.options import Options

#Options of Selenium to avoid chrome knowing you are a robot
opciones= Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])

opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--start-maximized') #empezar maximizado
opciones.add_argument('user.data-dir=selenium') #guarda las cookies
opciones.add_argument('drive/adblock.crx') #adblocker

from fake_useragent import UserAgent
usuario = UserAgent().random
opciones.add_argument(f'user-agent= {usuario}')

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.9/site-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/opt/homebrew/lib/python3.9/site-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


In [ ]:
driver.get('https://nomadlist.com/')

In [98]:
#scroll to botton of page to load all cities
# (https://www.codegrepper.com/search.php?q=selenium%20python%20scroll%20down%20until%20the%20end)
SCROLL_PAUSE_TIME = 1

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [101]:
ranks = driver.find_elements_by_xpath('//span[@class="rank"]')
city_names = driver.find_elements_by_xpath('//h2[@class="itemName"]')

city = []
for i in range(len(city_names)):
     city.append(city_names[i].text)

rank = []
for i in range(len(ranks)):
     rank.append(ranks[i].text)

#save to pandas df

In [ ]:
# cities web links
for i in city:
    

In [ ]:
for i in city:
    time.sleep(5)
    driver.get(f"https://nomadlist.com/{i}")
    t_score = driver.find_elements_by_xpath('//span[@xitemprop="ratingValue"]')
    print(t_score)

In [114]:
driver.get('https://nomadlist.com/porto')

In [137]:
num = driver.find_elements_by_xpath('//tr[@fdata-key="family_score"]')

numbr = []
for i in range(len(num)):
     numbr.append(num[i].text)

In [138]:
numbr

[]

In [ ]:
# find  all category elements 
total_score = driver.find_elements_by_xpath('//span[@xitemprop="ratingValue"]')